<a href="https://colab.research.google.com/github/Soham109/Big-5-personality-trait-detector/blob/main/technique_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets xgboost scikit-learn

import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from datasets import load_dataset

In [ ]:
big5 = load_dataset('Fatima0923/Automated-Personality-Prediction')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def get_pooled_embeddings(texts, model, tokenizer, max_length=512):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
    pooled_output = outputs.pooler_output
    return pooled_output.numpy()


In [ ]:
def extract_embeddings(dataset_split):
    texts = dataset_split['text']
    embeddings = get_pooled_embeddings(texts, bert_model, tokenizer)
    return embeddings

In [ ]:
train_embeddings = extract_embeddings(big5['train'])
validation_embeddings = extract_embeddings(big5['validation'])
test_embeddings = extract_embeddings(big5['test'])

In [ ]:
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
disc_df['embedding'] = disc_df['text_bio'].apply(lambda x: sbert_model.encode(x))

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
pca = PCA(n_components=5)
reduced_embeddings = pca.fit_transform(np.vstack(disc_df['embedding'].values))